In [25]:
import json
from functools import reduce
import numpy as np

filename = '/home/george/salience-prediction/datasets/reuters-128-entity-salience.ttl'

from rdflib import Graph
g = Graph()
g.parse(filename, format='turtle')


<Graph identifier=Nf1bfbdf803964f33ab98f58084eff138 (<class 'rdflib.graph.Graph'>)>

In [20]:
from rdflib import URIRef

def getSalienceScore(salience):
    if (salience == 'not_salient'):
        return 1
    if (salience == 'less_salient'):
        return 2
    if (salience == 'most_salient'):
        return 3
    
salience = URIRef('http://vse.cz/entity-salience#salience')
refContext = URIRef('http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#referenceContext')
refId = URIRef('http://www.w3.org/2005/11/its/rdf#taIdentRef')
refIsString = URIRef('http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#isString')
stories = {}

for entry in g.subject_objects(predicate=salience):
    (entity, score) = entry
    score = getSalienceScore(str(score))
    
    parentRef = g.value(entity, refContext)
    parentId = str(parentRef)
    
    idRef = g.value(entity, refId)
    id = str(idRef)
    
    if parentId not in stories.keys():
        content = g.value(subject=parentRef, predicate=refIsString)
        headline = ''
        body = str(content)
        stories[parentId] = {'entities' : {}, 'headline' : '', 'body' : body}
    
    story = stories[parentId]
    if id not in story.keys():
        story['entities'][id] = {'id' : str(id), 'forms' : []}
        
    entry = story['entities'][id]
    entry['score'] = score
    
    name = str(entity)
    offsets = list(map(int, name[name.find('=')+1:].split(',')))
    form = {'bodyOffsets' : [offsets], 'form' : stories[parentId]['body'][offsets[0]:offsets[1]]}
    entry['forms'].append(form)
        
        
for (suid, content) in stories.items():
    content['suid'] = suid
    content['entities'] = list(content['entities'].values())
    
stories = list(stories.values())
print(stories[0])
    


{'entities': [{'id': 'http://dbpedia.org/resource/Simulation', 'forms': [{'bodyOffsets': [[142, 152]], 'form': 'Simulation'}], 'score': 2}, {'id': 'http://dbpedia.org/resource/Purchasing', 'forms': [{'bodyOffsets': [[229, 238]], 'form': 'purchases'}], 'score': 3}, {'id': 'http://dbpedia.org/resource/Mln', 'forms': [{'bodyOffsets': [[101, 104]], 'form': 'mln'}], 'score': 1}, {'id': 'http://dbpedia.org/resource/Option', 'forms': [{'bodyOffsets': [[205, 211]], 'form': 'option'}], 'score': 2}, {'id': 'http://dbpedia.org/resource/Digital_Equipment_Corporation', 'forms': [{'bodyOffsets': [[18, 40]], 'form': 'Digital Equipment Corp'}], 'score': 3}, {'id': 'http://dbpedia.org/resource/Agreement', 'forms': [{'bodyOffsets': [[182, 191]], 'form': 'agreement'}], 'score': 3}, {'id': 'http://dbpedia.org/resource/Teradyne', 'forms': [{'bodyOffsets': [[0, 12]], 'form': 'Teradyne Inc'}], 'score': 3}, {'id': 'http://aksw.org/notInWiki/futrue_lasar', 'forms': [{'bodyOffsets': [[216, 228]], 'form': 'futru

In [24]:
with open('/home/george/salience-prediction/datasets/reuters-standard.json', 'w') as f:
    f.write(json.dumps(stories))
    
outdir = '/home/george/salience-prediction/datasets/reuters'
for i, story in enumerate(stories):
    outfile = outdir + '/' + str(i)
    with open(outfile, 'w') as f:
        f.write(story['body'])
    